## IMPORTS

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

In [4]:
plt.rcParams["figure.figsize"] = (20,15)

## DATA ANALYSIS

In [5]:
# Employee subery subset-3
df_emp_survey = pd.read_csv('subset-3-sous-ensemble-3.csv', encoding='ISO-8859-1')

# responses data from dicionary file responses
df_responses = pd.read_excel('2020-pses-supporting-documentation_document-de-reference-du-saff-2020.xlsx', sheet_name='RESPONSE OPTIONS DE RÉPONSES')

df_emp_survey.head()

,LEVEL1ID,LEVEL2ID,LEVEL3ID,LEVEL4ID,LEVEL5ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
0,0,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,75,73497,Public Service,Fonction publique,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
1,1,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,79,11550,Canada Revenue Agency,Agence du revenu du Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
2,2,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,80,5626,Employment and Social Development Canada,Emploi et Développement social Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
3,3,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,72,7308,Department of National Defence,Ministère de la Défense nationale,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
4,4,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,64,3252,Correctional Service Canada,Service correctionnel Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement


### DATA CLEANING

### Data Cleaning
- Remove ***French*** columns as they are 'duplicate' of english columns
- Remove Questions' ***Indicator ID & sub Indicator ID*** columns
- Remove ***level 2-5 id*** columns becuase they don't have data
- Remove all records with empty Answers 1-7 rows

### Levels 2-5 ID are all 0

In [6]:
all_levels = df_emp_survey[['LEVEL2ID', 'LEVEL3ID','LEVEL4ID','LEVEL5ID']]
non_zero_rows = len(all_levels[(all_levels != 0).all(axis=1)])
print(f'# rows where level 2-5 are not all empty:   {non_zero_rows}')
all_levels

# rows where level 2-5 are not all empty:   0


,LEVEL2ID,LEVEL3ID,LEVEL4ID,LEVEL5ID
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
1405574,0,0,0,0
1405575,0,0,0,0
1405576,0,0,0,0
1405577,0,0,0,0


## Delete rows that have no survey data (more than 50%) 1.4M -> 0.8M rows
- Some departments don't have certain equity groups. That means 0 in ANSWER 1-7 for non-exist equity group. 

In [19]:
all_answers = df_emp_survey[['ANSWER1', 'ANSWER2','ANSWER3','ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7']]
answered_rows = all_answers[(all_answers != ' ').all(axis=1)]
unAnswered_rows = all_answers[(all_answers == ' ').all(axis=1)]
set3_cleaned = df_emp_survey.iloc[answered_rows.index]
print(f'# rows that has no ANSWER 1-7:   {len(unAnswered_rows)}')
answered_rows
df_emp_survey.iloc[unAnswered_rows.index].head()
# set3_2020.to_csv('subset3_2020_cleaned.csv')

# rows that has no ANSWER 1-7:   822456


,LEVEL1ID,LEVEL2ID,LEVEL3ID,LEVEL4ID,LEVEL5ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
141,5,0,0,0,0,2020,Q115 = 3,Gender diverse,Genre divers,2013,...,,,Fisheries and Oceans Canada,Pêches et Océans Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
146,10,0,0,0,0,2020,Q115 = 3,Gender diverse,Genre divers,2013,...,,,"Innovation, Science and Economic Development C...","Innovation, Sciences et Développement économiq...",4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
148,12,0,0,0,0,2020,Q115 = 3,Gender diverse,Genre divers,2013,...,,,Transport Canada,Transports Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
150,14,0,0,0,0,2020,Q115 = 3,Gender diverse,Genre divers,2013,...,,,Global Affairs Canada,Affaires mondiales Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
151,15,0,0,0,0,2020,Q115 = 3,Gender diverse,Genre divers,2013,...,,,Natural Resources Canada,Ressources naturelles Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement


## Demographic Groups: 
- We have 59 demograohic groups
- This include non selected groups: eg. "Chinese" vs. "Chinese - Not selected"

In [17]:
# There are 59 equity groups
print(f'We have {len(df_emp_survey.DESCRIP_E.unique())} demographic groups')
list(df_emp_survey.DESCRIP_E.unique())

We have 59 demographic groups


['Male gender',
 'Female gender',
 'Gender diverse',
 'Indigenous',
 'Non-Indigenous',
 'First Nation (North American Indian)',
 'Métis',
 'Inuk (Inuit)',
 'Person with a disability',
 'Not a person with a disability',
 'A seeing disability affects vision, including total blindness, partial sight and visual distortion',
 'A seeing disability affects vision, including total blindness, partial sight and visual distortion - Not selected',
 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion',
 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion - Not selected',
 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility',
 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility - Not selected',


## Take year 2020 data
- 2020 has 330k rows
- 2019 has 176k rows
- 2018 has 76k rows

In [20]:
# print(set3.columns)
print(set3_cleaned.SURVEYR.unique())
print(f'2020: {len(set3_cleaned[set3_cleaned.SURVEYR==2020])} rows')
print(f'2019: {len(set3_cleaned[set3_cleaned.SURVEYR==2019])} rows')
print(f'2018: {len(set3_cleaned[set3_cleaned.SURVEYR==2018])} rows')

# have majority of data form 2020 so we focus on this year
set3_2020 = set3_cleaned[set3_cleaned.SURVEYR==2020]

[2020 2019 2018]
2020: 330739 rows
2019: 176361 rows
2018: 76023 rows


### number of equity groups in all 68 departments
- min = 2 (some department only have male & female?)
- max = 59 (all equity groups in the survey)

In [21]:
set3_2020['DEPT_E'].unique()
print('by # of equity groups surveyedin departments')
dept_equity = pd.Series(dtype=int)
for dept in set3_2020['DEPT_E'].unique():
    num_equity_groups = len(set3_2020[set3_2020['DEPT_E']==dept].DESCRIP_E.unique())
    dept_equity[dept] = num_equity_groups
dept_equity.describe()

by # of equity groups surveyedin departments


count    68.000000
mean     37.720588
std      16.127149
min       2.000000
25%      25.000000
50%      38.500000
75%      53.000000
max      59.000000
dtype: float64

### Look at the department that has only 2 equity groups
- Looks like it's just female & male
- We can't assume other things, but only the gender of surveyed people in that department
- but how is that possible when "Not Selected" is also an attribute value   -_-|||
- Gonna ignore the "not selected" groups

In [22]:
dept_2 = dept_equity[dept_equity == 2].index[0]
print(dept_2)
set3_2020[set3_2020['DEPT_E']==dept_2]['DESCRIP_E'].unique()

Canadian Security Intelligence Service


array(['Male gender', 'Female gender'], dtype=object)

## PROBLEM ANALYSIS & VISUALIZATION

## CONCLUSIONS & RECOMENDATIONS